In [1]:
#downloading and loading query results from Amazon Athena into a Pandas DataFrame. 

import time
import pandas as pd
import boto3
from typing import Dict

# Define your AWS credentials and configuration

AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = ""
SCHEMA_NAME = "covid_19"
S3_STAGING_DIR = "s3://sal-covid19-ds/output/"
S3_BUCKET_NAME = "sal-covid19-ds"
S3_OUTPUT_DIRECTORY = "output"



# Initialize your Athena client
athena_client = boto3.client('athena', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=AWS_REGION)

def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            # Poll the query execution status
            query_execution = client.get_query_execution(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            status = query_execution['QueryExecution']['Status']['State']

            if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break

            # Wait and poll again if the query is still running
            time.sleep(5)
        except Exception as err:
            raise err

    # Check if the query was successful
    if status == 'SUCCEEDED':
        # Download the result data from S3
        temp_file_location = "athena_query_results.csv"
        s3_client = boto3.client(
            "s3",
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY,
            region_name=AWS_REGION,
        )
        s3_client.download_file(
            S3_BUCKET_NAME,
            f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
            temp_file_location,
        )

        # Load the CSV data into a Pandas DataFrame
        return pd.read_csv(temp_file_location)
    else:
        raise Exception("Athena query execution failed.")

# Example usage
response = athena_client.start_query_execution(
    QueryString='SELECT * FROM "enigma-nytimes-data-in-usaus_county"',
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

enigma_nytimes_data_in_usaus_county = download_and_load_query_results(athena_client, response)


In [43]:
factCovid1 = enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid2 = rearc_covid_19_testing_datastates_daily[['fips','date','positive','negative','hospitalized','hospitalizedcurrently','hospitalizeddischarged']]


factCovid02 = pd.merge(factCovid1, factCovid2, on='fips', how='inner')

In [70]:
factCovid.shape

(6683, 13)

In [53]:
factCovid1 = enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]

factCovid1

,fips,province_state,country_region,confirmed,deaths,recovered,active
0,NaN,Anhui,China,1.0,NaN,NaN,NaN
1,NaN,Beijing,China,14.0,NaN,NaN,NaN
2,NaN,Chongqing,China,6.0,NaN,NaN,NaN
3,NaN,Fujian,China,1.0,NaN,NaN,NaN
4,NaN,Gansu,China,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
39995,29189.0,Missouri,US,605.0,3.0,0.0,0.0
39996,29510.0,Missouri,US,195.0,1.0,0.0,0.0
39997,12111.0,Florida,US,40.0,2.0,0.0,0.0
39998,22099.0,Louisiana,US,41.0,3.0,0.0,0.0


In [54]:
factCovid2 = rearc_covid_19_testing_datastates_daily[['fips','date','positive','negative','hospitalized','hospitalizedcurrently','hospitalizeddischarged']]

factCovid2

,fips,date,positive,negative,hospitalized,hospitalizedcurrently,hospitalizeddischarged
0,2.0,20210307,56886,NaN,1293.0,33.0,NaN
1,1.0,20210307,499819,1931711.0,45976.0,494.0,NaN
2,5.0,20210307,324818,2480716.0,14926.0,335.0,NaN
3,60.0,20210307,0,2140.0,NaN,NaN,NaN
4,4.0,20210307,826454,3073010.0,57907.0,963.0,118932.0
...,...,...,...,...,...,...,...
2680,49.0,20210119,326221,1385896.0,12645.0,638.0,NaN
2681,51.0,20210119,451076,NaN,20066.0,3173.0,35752.0
2682,78.0,20210119,2260,37119.0,NaN,NaN,NaN
2683,50.0,20210119,10321,280334.0,NaN,43.0,NaN


In [56]:
factCovid.shape

(6683, 13)

In [57]:
dimRegion_1 = enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
dimRegion_2 = enigma_nytimes_data_in_usaus_county[['fips','county','state']]
dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')

In [59]:
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [62]:
dimHospital = rearc_usa_hospital_bedsusa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [63]:
dimHospital.head()

,fips,state_name,latitude,longtitude,hq_address,hospital_name,hospital_type,hq_city,hq_state


In [64]:
dimDate = rearc_covid_19_testing_datastates_daily[['fips','date']]
dimDate

,fips,date
0,2.0,20210307
1,1.0,20210307
2,5.0,20210307
3,60.0,20210307
4,4.0,20210307
...,...,...
2680,49.0,20210119
2681,51.0,20210119
2682,78.0,20210119
2683,50.0,20210119


In [66]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')

C:\Users\lasis\AppData\Local\Temp\ipykernel_39268\572748324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [67]:
dimDate.head()

,fips,date
0,2.0,2021-03-07
1,1.0,2021-03-07
2,5.0,2021-03-07
3,60.0,2021-03-07
4,4.0,2021-03-07


In [68]:
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek

C:\Users\lasis\AppData\Local\Temp\ipykernel_39268\935310350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\lasis\AppData\Local\Temp\ipykernel_39268\935310350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month
C:\Users\lasis\AppData\Local\Temp\ipykernel_39268\935310350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [69]:
dimDate

,fips,date,year,month,day_of_week
0,2.0,2021-03-07,2021,3,6
1,1.0,2021-03-07,2021,3,6
2,5.0,2021-03-07,2021,3,6
3,60.0,2021-03-07,2021,3,6
4,4.0,2021-03-07,2021,3,6
...,...,...,...,...,...
2680,49.0,2021-01-19,2021,1,1
2681,51.0,2021-01-19,2021,1,1
2682,78.0,2021-01-19,2021,1,1
2683,50.0,2021-01-19,2021,1,1
